In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ETA_Regression_Machinelearning") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()

spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/15 01:17:32 WARN Utils: Your hostname, DESKTOP-T1QBIQ7, resolves to a loopback address: 127.0.1.1; using 192.168.100.144 instead (on interface wifi0)
26/01/15 01:17:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/15 01:17:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.parquet(
    "/home/khadija/spark-warehouse/silver.db/silver_taxi"
)

df.printSchema()
df.show(5)
df.count()



root
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- cbd_congestion_fee: double (nullable = true)
 |-- trip_duree: double (nullable = true)
 |-- pickup_hour: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- month: integer (nullable = true)



+-------------+----------+------------+-----------+----------+------------+------------------+------------------+-----------+-----------+-----+
|trip_distance|RatecodeID|payment_type|fare_amount|tip_amount|total_amount|cbd_congestion_fee|        trip_duree|pickup_hour|day_of_week|month|
+-------------+----------+------------+-----------+----------+------------+------------------+------------------+-----------+-----------+-----+
|         1.07|         1|           1|       16.3|      4.21|       25.26|              0.75|19.633333333333333|         18|          1|    1|
|         1.21|         1|           1|        9.3|      2.81|       16.86|              0.75| 8.516666666666667|         18|          1|    1|
|          2.0|         1|           1|       14.2|      3.75|        22.7|              0.75|14.016666666666667|         18|          1|    1|
|          2.3|         1|           1|       14.9|       4.9|       24.55|              0.75|13.666666666666666|         18|          1

2403477

In [5]:
from pyspark.ml.feature import VectorAssembler
target = "trip_duree"

# Step 4: Feature engineering (Build the feature vector)
feature_columns = ["trip_distance","fare_amount","tip_amount","total_amount","cbd_congestion_fee","pickup_hour","day_of_week","month","RatecodeID","payment_type"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

In [22]:
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

def training_evaluation_model(df, assembler, target, model, split=[0.8, 0.2]):
 
   
    # Créer pipeline
    pipeline = Pipeline(stages=[assembler, model])
    
    # Split data
    train_data, test_data = df.randomSplit(split, seed=42)
    
    # Entraîner modèle
    trained_model = pipeline.fit(train_data)
    
    # Prédictions
    predictions = trained_model.transform(test_data)
    
    # Évaluation
    evaluator_r2 = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="r2")
    evaluator_rmse = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
    
    r2 = evaluator_r2.evaluate(predictions)
    rmse = evaluator_rmse.evaluate(predictions)
    
    print(f"Modèle: {model.__class__.__name__}")
    print(f"R²: {r2:.4f} | RMSE: {rmse:.4f}")
    
    return {
        "model": trained_model,
        "r2": r2,
        "rmse": rmse,
        "predictions": predictions
    }


In [ ]:
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor

models = [
    LinearRegression(featuresCol="features", labelCol=target),
    RandomForestRegressor(featuresCol="features", labelCol=target),
    GBTRegressor(featuresCol="features", labelCol=target)
]


# Tester tous les modèles et afficher directement R² et RMSE
for m in models:
    res = training_evaluation_model(df, assembler, target, m)
    


26/01/15 02:49:33 WARN Instrumentation: [27795430] regParam is zero, which might cause numerical instability and overfitting.


Modèle: LinearRegression
R²: 0.8811 | RMSE: 2.4677


26/01/15 02:50:14 WARN MemoryStore: Not enough space to cache rdd_864_2 in memory! (computed 40.6 MiB so far)
26/01/15 02:50:14 WARN BlockManager: Persisting block rdd_864_2 to disk instead.
26/01/15 02:50:14 WARN MemoryStore: Not enough space to cache rdd_864_0 in memory! (computed 27.1 MiB so far)
26/01/15 02:50:14 WARN BlockManager: Persisting block rdd_864_0 to disk instead.
26/01/15 02:50:17 WARN MemoryStore: Not enough space to cache rdd_864_1 in memory! (computed 93.8 MiB so far)
26/01/15 02:50:17 WARN BlockManager: Persisting block rdd_864_1 to disk instead.
26/01/15 02:50:25 WARN MemoryStore: Not enough space to cache rdd_864_1 in memory! (computed 140.7 MiB so far)
26/01/15 02:50:25 WARN MemoryStore: Not enough space to cache rdd_864_0 in memory! (computed 140.7 MiB so far)
26/01/15 02:50:32 WARN MemoryStore: Not enough space to cache rdd_864_1 in memory! (computed 140.7 MiB so far)
26/01/15 02:50:32 WARN MemoryStore: Not enough space to cache rdd_864_0 in memory! (computed 1

Modèle: RandomForestRegressor
R²: 0.8826 | RMSE: 2.4521


26/01/15 02:51:35 WARN MemoryStore: Not enough space to cache rdd_924_1 in memory! (computed 29.5 MiB so far)
26/01/15 02:51:35 WARN BlockManager: Persisting block rdd_924_1 to disk instead.
26/01/15 02:51:35 WARN MemoryStore: Not enough space to cache rdd_924_2 in memory! (computed 44.2 MiB so far)
26/01/15 02:51:35 WARN BlockManager: Persisting block rdd_924_2 to disk instead.


Modèle: GBTRegressor
R²: 0.9299 | RMSE: 1.8950
